In [ ]:
def replace_langda_and_lann_terms(text_list:List[Tuple[str, str, str]]) -> Tuple[List[str],List[dict],List[dict]]:
    """
    Replace langda and lann predicates in the given text list.
    And store informations in dictonary
    Args:
        text_list: List of tuples (code, comment, predicate_status)
                   where predicate_status can be "NONE", "BODY", or "END."
    It calls the function: parse_lann_or_langda_content_to_dicts()
    Returns:
        Tuple of (modified text list, lann_dicts, langda_dicts)
    """
    print("processing langda and lann terms...")
    # Initialize outputs
    text_list_copy = text_list.copy()  # Create a copy to avoid modifying the original
    lann_dicts:List[dict] = []
    langda_dicts:List[dict] = []
    result_text_list = []

    # State variables
    single_langda = []
    single_lann = []
    single_comment = []
    in_langda = False
    in_lann = False
    langda_start = -1
    lann_start = -1
    
    idl = 0
    while idl < len(text_list_copy):
        current_item:Tuple[str,str,str] = text_list_copy[idl]
        if len(current_item) != 3:
            raise ValueError(f"Warning: Position {idl} has a uncorrect form: {current_item}")
            
        (code, comment, predicate_status) = current_item
        has_langda = code.startswith("langda(")
        has_lann = code.startswith("lann(")

        # -------------------- #           PART1            # -------------------- #
        # -------------------- Process single lann predicates -------------------- #
        # Start of lann predicate
        if has_lann and not in_lann:
            in_lann = True
            lann_start = idl    # mark the start of lann predicate
            single_lann = []
            single_comment = []
            if not idl + 1 < len(text_list_copy):
                raise ValueError("The code is incomplete, please check your lann predicates.")


        # -------------------- #            PART2             # -------------------- #
        # -------------------- Process single langda predicates -------------------- #
        # Start of langda predicate
        if has_langda and not in_langda:
            in_langda = True
            langda_start = idl     # mark the start of langda predicate
            single_langda = []
            single_comment = []


        if in_lann:
            # Middle of lann predicate
            if predicate_status == "BODY":
                single_lann.append(code)
                single_comment.append(comment)

            # End of lann predicate
            elif predicate_status == "END.":
                # Add the current segment
                single_lann.append(code)
                single_comment.append(comment)
                
                # Create the full lann term and its dict representation
                full_lann_term = "".join(single_lann)
                full_lann_content = full_lann_term[5:-1]
                lann_dict = parse_lann_or_langda_content_to_dicts(full_lann_content)
                lann_dicts.append(lann_dict)

                # -------------------- # REPLACE # -------------------- #
                # Replace the original segments with our processed version:
                # nn(net_name,[X],Y,[1,2,3])::digit(X,Y).
                nn_term = f"nn({','.join([k for k in lann_dict.keys()])})"
                
                # Filter out empty comments before joining (optional, same as langda)
                filtered_comments = [c for c in single_comment if c]
                joined_comments = "\n".join(filtered_comments) + "\n" + nn_term
                
                # print("#####nn_term:", nn_term)
                # print("#####full_lann_term:", full_lann_term)
                # print("#####full_lann_content:", full_lann_content) 
                # print("#####lann_dict:", lann_dict)
                # print("#####single_lann:", single_lann)
                # print("#####single_comment:", single_comment)
                # print("#####filtered_comments:", filtered_comments)
                # print("#####start,idl", lann_start, idl)
                # print("#####[lann_start:idl]:", text_list_copy[lann_start:idl+1])
                
                # Replace all items from lann_start to idl with a single item containing our joined comments and term
                text_list_copy[lann_start:idl+1] = [(joined_comments, "", "NONE")]

                # Adjust index to continue after our replacement - always advance by 1 
                # since we replaced with a single item
                idl = lann_start

                # Reset state
                lann_dict = {}
                in_lann = False


        elif in_langda:
            # Middle of langda predicate
            if predicate_status == "BODY":
                single_langda.append(code)
                single_comment.append(comment)
            
            # End of langda predicate
            elif predicate_status == "END.":
                # Add the current segment
                single_langda.append(code)
                single_comment.append(comment)
                
                # Create the full langda term and its dict representation
                full_langda_term = " ".join(single_langda)
                full_langda_content = full_langda_term[7:-1]
                langda_dict = parse_lann_or_langda_content_to_dicts(full_langda_content)
                langda_dicts.append(langda_dict)
                
                # -------------------- # REPLACE # -------------------- #
                # Replace the original segments with our processed version:
                # # comment1...
                # # comment2...
                #  "\n{{LANGDA}}\n"
                
                # Filter out empty comments before joining
                filtered_comments = [c for c in single_comment if c]
                joined_comments = "\n".join(filtered_comments) + "\n{{LANGDA}}"
                
                # print("#####full_langda_term:", full_langda_term)
                # print("#####full_langda_content:", full_langda_content)
                # print("#####langda_dict:", langda_dict)
                # print("#####single_langda:", single_langda)
                # print("#####single_comment:", single_comment)
                # print("#####filtered_comments:", filtered_comments)
                # print("#####start,idl", langda_start, idl)
                # print("#####[langda_start:idl]:", text_list_copy[langda_start:idl+1])
                
                # Replace all items from langda_start to idl with a single item containing our joined comments
                text_list_copy[langda_start:idl+1] = [(joined_comments, "", "NONE")]
                
                # Adjust index to continue after our replacement - always advance by 1
                # since we replaced with a single item
                idl = langda_start
                
                # Reset state
                langda_dict = {}
                in_langda = False

        else:
            pass
        idl += 1
    
    return [item[0] for item in text_list_copy], lann_dicts, langda_dicts